**LSA Algorithm**

In [ ]:
from operator import attrgetter
from collections import namedtuple
#from utils import ItemsCount

SentenceInfo = namedtuple("SentenceInfo", ("sentence", "order", "rating",))

class ItemsCount(object):
    def __init__(self, value):
        self._value = value

    def __call__(self, sequence):
        if isinstance(self._value, (bytes, str,)):
            if self._value.endswith("%"):
                total_count = len(sequence)
                percentage = int(self._value[:-1])
                # at least one sentence should be chosen
                count = max(1, total_count*percentage // 100)
                return sequence[:count]
            else:
                return sequence[:int(self._value)]
        elif isinstance(self._value, (int, float)):
            return sequence[:int(self._value)]
        else:
            ValueError("Unsuported value of items count '%s'." % self._value)

    def __repr__(self):
        return to_string("<ItemsCount: %r>" % self._value)

class BaseSummarizer(object):
    
    def __call__(self, document, sentences_count):
        raise NotImplementedError("This method should be overriden in subclass")

    @staticmethod
    def normalize_word(word):
        return word.lower()

    @staticmethod
    def _get_best_sentences(sentences, count, rating, *args, **kwargs):
        rate = rating
        if isinstance(rating, dict):
            assert not args and not kwargs
            rate = lambda s: rating[s]

        infos = (SentenceInfo(s, o, rate(s, *args, **kwargs))
            for o, s in enumerate(sentences))

        # sort sentences by rating in descending order
        infos = sorted(infos, key=attrgetter("rating"), reverse=True)
        # get `count` first best rated sentences
        if not isinstance(count, ItemsCount):
            count = ItemsCount(count)
        infos = count(infos)
        # sort sentences by their order in document
        infos = sorted(infos, key=attrgetter("order"))

        return tuple(i.sentence for i in infos)

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
import math
import numpy
import nltk

from warnings import warn
from nltk.tokenize import sent_tokenize, word_tokenize
from numpy.linalg import svd as singular_value_decomposition
#from base_summarizer import BaseSummarizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

class LsaSummarizer(BaseSummarizer):
    MIN_DIMENSIONS = 3
    REDUCTION_RATIO = 1/1

    _stop_words = list(stopwords.words('english'))

    @property
    def stop_words(self):
        return self._stop_words

    @stop_words.setter
    def stop_words(self, words):
        self._stop_words = words

    def __call__(self, document, sentences_count):

        dictionary = self._create_dictionary(document)
        
        if not dictionary:
            return ()

        sentences = sent_tokenize(document)

        matrix = self._create_matrix(document, dictionary)
        matrix = self._compute_term_frequency(matrix)
        u, sigma, v = singular_value_decomposition(matrix, full_matrices=False)

        ranks = iter(self._compute_ranks(sigma, v))
        return self._get_best_sentences(sentences, sentences_count,
            lambda s: next(ranks))

    def _create_dictionary(self, document):
        """Creates mapping key = word, value = row index"""

        words = word_tokenize(document)
        words = tuple(words)

        words = map(self.normalize_word, words)

        unique_words = frozenset(w for w in words if w not in self._stop_words)

        return dict((w, i) for i, w in enumerate(unique_words))

    def _create_matrix(self, document, dictionary):
        """
        """
        sentences = sent_tokenize(document)
        words_count = len(dictionary)
        sentences_count = len(sentences)
        if words_count < sentences_count:
            message = (
                "Number of words (%d) is lower than number of sentences (%d). "
                "LSA algorithm may not work properly."
            )
            warn(message % (words_count, sentences_count))

        matrix = numpy.zeros((words_count, sentences_count))
        for col, sentence in enumerate(sentences):
            words = word_tokenize(sentence)
            for word in words:
                # only valid words is counted (not stop-words, ...)
                if word in dictionary:
                    row = dictionary[word]
                    matrix[row, col] += 1

        return matrix

    def _compute_term_frequency(self, matrix, smooth=0.4):
        """
        """
        assert 0.0 <= smooth < 1.0

        max_word_frequencies = numpy.max(matrix, axis=0)
        rows, cols = matrix.shape
        for row in range(rows):
            for col in range(cols):
                max_word_frequency = max_word_frequencies[col]
                if max_word_frequency != 0:
                    frequency = matrix[row, col]/max_word_frequency
                    matrix[row, col] = smooth + (1.0 - smooth)*frequency

        return matrix

    def _compute_ranks(self, sigma, v_matrix):
        assert len(sigma) == v_matrix.shape[0]

        dimensions = max(LsaSummarizer.MIN_DIMENSIONS,
            int(len(sigma)*LsaSummarizer.REDUCTION_RATIO))
        powered_sigma = tuple(s**2 if i < dimensions else 0.0
            for i, s in enumerate(sigma))

        ranks = []
        
        for column_vector in v_matrix.T:
            rank = sum(s*v**2 for s, v in zip(powered_sigma, column_vector))
            ranks.append(math.sqrt(rank))

        return ranks


#from lsa_summarizer import LsaSummarizer
import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

from nltk.corpus import stopwords


summarizer = LsaSummarizer()
text='''
Extracting relevant feature and classification are significant in brain-computer interface (BCI) systems. Deep learning have achieved remarkable growth in many fields like speech recognition and computer vision. However, deep learning in biomedical field is yet to be fully utilized. In this paper, We propose a novel methodology for convolutional neural network (CNN) based motor imagery (MI) classification using new form of input. Continuous Wavelet Transform (CWT) is applied to the input Electroencephalography (EEG) signal to extract the features of MI. After transformation, we consider the real part and imaginary part of the transformed signal to exploit magnitude and phase information at the same time. This feature is fed to the CNN having one convolution layer, one max-pooling layer and one fully connected layer. The classification accuracy is tested on two public BCI datasets: BCI competition IV dataset IIb and BCI competition II dataset III. The proposed method shows increase in classification accuracy compared to other MI classification methods. The results show that the method using CNN with magnitude and phase based features can be better than other state-of-the-art approaches.
'''

stopwords = stopwords.words('english')
summarizer.stop_words = stopwords
summary =summarizer(text, 5)
string = ''.join(summary)

print("\n========= Summary =========")
print(string)
print("========= End of summary =========")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.

========= Summary =========

Extracting relevant feature and classification are significant in brain-computer interface (BCI) systems.Deep learning have achieved remarkable growth in many fields like speech recognition and computer vision.However, deep learning in biomedical field is yet to be fully utilized.In this paper, We propose a novel methodology for convolutional neural network (CNN) based motor imagery (MI) classification using new form of input.The results show that the method using CNN with magnitude and phase based features can be better than other state-of-the-art approaches.
========= End of summary =========


In [ ]:
text1='''
The Fifth generation (5G) network is projected to support large amount of data traffic and massive number of wireless connections. Different data traffic has different Quality of Service (QoS) requirements. 5G mobile network aims to address the limitations of previous cellular standards (i.e., 2G/3G/4G) and be a prospective key enabler for future Internet of Things (IoT). 5G networks support a wide range of applications such as smart home, autonomous driving, drone operations, health and mission critical applications, Industrial IoT (IIoT), and entertainment and multimedia. Based on end users’ experience, several 5G services are categorized into immersive 5G services, intelligent 5G services, omnipresent 5G services, autonomous 5G services, and public 5G services. In this paper, we present a brief overview of 5G technical scenarios. We then provide a brief overview of accepted papers in our Special Issue on 5G mobile services and scenarios. Finally, we conclude this paper.
'''
ref1='''
5G network is projected to support large amount of data traffic and massive number of wireless connections. 5G networks support a wide range of applications such as smart home, autonomous driving, drone operations, health and mission critical applications.
'''

text2='''
The modern technology demands the maintenance of the increasing data which are in structured and unstructured form. The text documents collected in the various platforms occupies a massive space in the architectural structure of the computer system both physically and virtually. Apparently the users demand the summarizing of the collected documents for easy access and usage. To enable this automatic text summarization came into phase. The automatic text summarization condenses the text documents into meaningful phrases and textual messages which helps the user to understand the conceptual ides behind each core values. The importance of automatic text summarization stands as a helping source in the growing data. This paper discusses the basic blocks of the automatic text summarization and its feature in identifying the intricate properties of the meaningful text through various approaches.
'''
ref2='''
Automatic text summarization condenses the text documents into meaningful phrases and textual messages. This paper discusses the basic blocks of the automatic text summarizing and its feature in identifying the intricate properties.
'''

text3='''
We present an Integer Linear Program for exact inference under a maximum coverage model for automatic summarization. We compare our model, which operates at the sub-sentence or “concept”-level, to a sentence-level model, previously solved with an ILP. Our model scales more efficiently to larger problems because it does not require a quadratic number of variables to address redundancy in pairs of selected sentences. We also show how to include sentence compression in the ILP formulation, which has the desirable property of performing compression and sentence selection simultaneously. The resulting system performs at least as well as the best systems participating in the recent Text Analysis Conference, as judged by a variety of automatic and manual content-based metrics.
'''
ref3='''
We present a model for exact inference under a maximum coverage model for automatic summarization. We compare our model, which operates at the sub-sentence or "concept"-level, to a sentence-level model. Our model scales more efficiently to larger problems because it does not require a quadratic number of variables.
'''

text4='''
In this paper, we propose a universal solution to web search and web browsing on handheld devices for visually impaired people. For this purpose, we propose (1) to automatically cluster web page results and (2) to summarize all the information in web pages so that speech-to-speech interaction is used efficiently to access information.
'''
ref4='''
In this paper, we propose a universal solution to web search and web browsing on handheld devices for visually impaired people.
'''

text5='''
Automatic text summarization (ATS) is the process of generating a summary by condensing text document by a computer machine. In this paper, we explored voting-based extractive approaches for text summarization. The main issue with most of the feature-based ATS methods is to find optimal feature weights for sentence scoring to optimize the quality of summary. Voting-based methods are sensitive to initial ranking process. We proposed reciprocal ranking-based sentence scoring approach that alleviates the feature weighting and initial ranking problem. The proposed approach uses a specific prominent set of features for initial ranking that further enhance the performance. Experimental results on Document Understating Conference 2002 data-set using ROUGE evaluation matrices shows that our proposed method performs better as compared to other voting-based methods.
'''
ref5='''
Automatic text summarization is the process of generating a summary by condensing text document by a computer machine. Main issue with most of the feature-based ATS methods is to find optimal feature weights for sentence scoring to optimize the quality of summary.
'''

text6='''
Vehicular Ad-Hoc Networks (VANET) are considered as a subset of Mobile Ad-Hoc Networks (MANET). VANET is mainly used for the construction of an intelligent transport system. VANET enables communication between the vehicles (V2V) and vehicles to infrastructure (V2I). VANET can be used to coordinate the traffic, improve safety measures, support the drivers for hassle-free driving. It plays a major role in building smart cities in the near future. VANET is vulnerable to a number of security issues among which the DoS attack is a major part. DoS attack in VANET involves a malicious node flooding a huge amount of traffic using spoofed identities. This, in turn, may disrupt the services of vehicles in the network. The detection of the attack becomes very difficult due to fake identities. The detection scheme uses a cuckoo filter and IP detection technique to detect the attack in the network. Once the attack is detected it generates a broadcast message to all the other vehicles that are present in the network.
'''
ref6='''
VANET is a subset of Mobile Ad-Hoc Networks (MANET) It enables communication between the vehicles (V2V) and vehicles to infrastructure. VANET can be used to coordinate the traffic, improve safety measures, support the drivers for hassle-free driving.
'''

text7='''
Face identification is the problem of determining whether two face images depict the same person or not. This is difficult due to variations in scale, pose, lighting, background, expression, hairstyle, and glasses. In this paper we present two methods for learning robust distance measures: (a) a logistic discriminant approach which learns the metric from a set of labelled image pairs (LDML) and (b) a nearest neighbour approach which computes the probability for two images to belong to the same class (MkNN). We evaluate our approaches on the Labeled Faces in the Wild data set, a large and very challenging data set of faces from Yahoo! News. The evaluation protocol for this data set defines a restricted setting, where a fixed set of positive and negative image pairs is given, as well as an unrestricted one, where faces are labelled by their identity. We are the first to present results for the unrestricted setting, and show that our methods benefit from this richer training data, much more so than the current state-of-the-art method. Our results of 79.3% and 87.5% correct for the restricted and unrestricted setting respectively, significantly improve over the current state-of-the-art result of 78.5%. Confidence scores obtained for face identification can be used for many applications e.g. clustering or recognition from a single training example. We show that our learned metrics also improve performance for these tasks.
'''
ref7='''
Face identification is the problem of determining whether two face images depict the same person or not. This is difficult due to variations in scale, pose, lighting, background, expression, hairstyle, and glasses. In this paper we present two methods for learning robust distance measures. We show our methods benefit from richer training data, much more so than the current state-of-the-art method.
'''

text8='''
Identification of different risk factors and early prediction of mortality for patients with heart failure are crucial for guiding clinical decision-making in Intensive care unit cohorts. In this paper, we developed a comprehensive risk model for predicting heart failure mortality with a high level of accuracy using an improved random survival forest (iRSF). Utilizing a novel split rule and stopping criterion, the proposed iRSF was able to identify more accurate predictors to separate survivors and nonsurvivors and thus improve discrimination ability. Based on the public MIMIC II clinical database with 8 059 patients, 32 risk factors, including demographics, clinical, laboratory information, and medications, were analyzed and used to develop the risk model for patients with heart failure. Compared with previous studies, more critical laboratory predictors were identified that could reveal difficult-to-manage comorbidities, including aspartate aminotransferase, alanine aminotransferase, total bilirubin, serum creatine, blood urea nitrogen, and their inherent effects on events; these were determined to be critical indicators for predicting heart failure mortality with the proposed iRSF. The experimental results showed that the developed risk model was superior to those used in previous studies and the conventional random survival forest-based model with an out-of-bag C-statistic value of 0.821. Therefore, the developed iRSF-based risk model could serve as a valuable tool for clinicians in heart failure mortality prediction.
'''
ref8='''
Early prediction of mortality for patients with heart failure is crucial for guiding clinical decision-making. In this paper, we developed a risk model for predicting heart failure mortality with a high level of accuracy using an improved random survival forest (iRSF) The developed iRSF-based risk model could serve as a valuable tool for clinicians.
'''

text9='''
Naive Bayes is one of the most widely used algorithms in classification problems because of its simplicity, effectiveness, and robustness. It is suitable for many learning scenarios, such as image classification, fraud detection, web mining, and text classification. Naive Bayes is a probabilistic approach based on assumptions that features are independent of each other and that their weights are equally important. However, in practice, features may be interrelated. In that case, such assumptions may cause a dramatic decrease in performance. In this study, by following preprocessing steps, a Feature Dependent Naive Bayes (FDNB) classification method is proposed. Features are included for calculation as pairs to create dependence between one another. This method was applied to the software defect prediction problem and experiments were carried out using widely recognized NASA PROMISE data sets. The obtained results show that this new method is more successful than the standard Naive Bayes approach and that it has a competitive performance with other feature-weighting techniques. A further aim of this study is to demonstrate that to be reliable, a learning model must be constructed by using only training data, as otherwise misleading results arise from the use of the entire data set.
'''
ref9='''
Naive Bayes is a probabilistic approach based on assumptions that features are independent of each other. It is suitable for many learning scenarios, such as image classification, fraud detection, web mining, and text classification. In this study, by following preprocessing steps, a Feature Dependent Naives (FDNB) classification method is proposed.
'''

text10='''
Extracting relevant feature and classification are significant in brain-computer interface (BCI) systems. Deep learning have achieved remarkable growth in many fields like speech recognition and computer vision. However, deep learning in biomedical field is yet to be fully utilized. In this paper, We propose a novel methodology for convolutional neural network (CNN) based motor imagery (MI) classification using new form of input. Continuous Wavelet Transform (CWT) is applied to the input Electroencephalography (EEG) signal to extract the features of MI. After transformation, we consider the real part and imaginary part of the transformed signal to exploit magnitude and phase information at the same time. This feature is fed to the CNN having one convolution layer, one max-pooling layer and one fully connected layer. The classification accuracy is tested on two public BCI datasets: BCI competition IV dataset IIb and BCI competition II dataset III. The proposed method shows increase in classification accuracy compared to other MI classification methods. The results show that the method using CNN with magnitude and phase based features can be better than other state-of-the-art approaches.
'''
ref10='''
Deep learning in biomedical field is yet to be fully utilized. Method using CNN with magnitude and phase based features can be better than other state-of-the-art approaches. The proposed method shows increase in classification accuracy compared to other MI classification methods.
'''



In [ ]:
from nltk.translate.bleu_score import sentence_bleu

candidate = string.split()
print('BLEU score -> {}'.format(sentence_bleu(ref10, string )))


print('Individual 1-gram: %f' % sentence_bleu(ref10, string, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(ref10, string, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(ref10, string, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(ref10, string, weights=(0, 0, 0, 1)))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU score -> 0.48355364211643287
Individual 1-gram: 0.054674
Individual 2-gram: 1.000000
Individual 3-gram: 1.000000
Individual 4-gram: 1.000000


In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge 
rouge = Rouge()
scores = rouge.get_scores(ref10, string)
print(scores)


[{'rouge-1': {'f': 0.4959999955379201, 'p': 0.7380952380952381, 'r': 0.37349397590361444}, 'rouge-2': {'f': 0.39024389799458, 'p': 0.5853658536585366, 'r': 0.2926829268292683}, 'rouge-l': {'f': 0.5045045000600602, 'p': 0.7567567567567568, 'r': 0.3783783783783784}}]
